In [6]:
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.static import players
from nba_api.stats.static import teams

In [7]:
# Find players by full name.
players.find_players_by_full_name('james')

[{'first_name': 'James',
  'full_name': 'James Anderson',
  'id': 202341,
  'is_active': False,
  'last_name': 'Anderson'},
 {'first_name': 'James',
  'full_name': 'James Augustine',
  'id': 200788,
  'is_active': False,
  'last_name': 'Augustine'},
 {'first_name': 'James',
  'full_name': 'James Bailey',
  'id': 76085,
  'is_active': False,
  'last_name': 'Bailey'},
 {'first_name': 'James',
  'full_name': 'James Blackwell',
  'id': 1806,
  'is_active': False,
  'last_name': 'Blackwell'},
 {'first_name': 'James',
  'full_name': 'James Collins',
  'id': 1530,
  'is_active': False,
  'last_name': 'Collins'},
 {'first_name': 'James',
  'full_name': 'James Cotton',
  'id': 1526,
  'is_active': False,
  'last_name': 'Cotton'},
 {'first_name': 'JamesOn',
  'full_name': 'JamesOn Curry',
  'id': 201191,
  'is_active': False,
  'last_name': 'Curry'},
 {'first_name': 'James',
  'full_name': 'James Davis',
  'id': 76524,
  'is_active': False,
  'last_name': 'Davis'},
 {'first_name': 'James',
  'fu

In [8]:
players.find_players_by_full_name('luka')

[{'first_name': 'Luka',
  'full_name': 'Luka Doncic',
  'id': 1629029,
  'is_active': True,
  'last_name': 'Doncic'},
 {'first_name': 'Luka',
  'full_name': 'Luka Samanic',
  'id': 1629677,
  'is_active': True,
  'last_name': 'Samanic'}]

In [9]:
# Find teams by full name.
teams.find_teams_by_full_name('cav')

[{'abbreviation': 'CLE',
  'city': 'Cleveland',
  'full_name': 'Cleveland Cavaliers',
  'id': 1610612739,
  'nickname': 'Cavaliers',
  'state': 'Ohio',
  'year_founded': 1970}]

In [10]:
player_id = 1629029
player_info = commonplayerinfo.CommonPlayerInfo(player_id=player_id)
player_id

1629029

In [11]:
player_info.available_seasons.get_json()

'{"headers": ["SEASON_ID"], "data": [["12018"], ["22018"], ["12019"], ["22019"]]}'

In [12]:
player_info.available_seasons.get_dict()


{'data': [['12018'], ['22018'], ['12019'], ['22019']],
 'headers': ['SEASON_ID']}

In [13]:
# Returns the data set in a pandas DataFrame.
player_info.available_seasons.get_data_frame()

,SEASON_ID
0,12018
1,22018
2,12019
3,22019


In [14]:
# Basic Request
player_info = commonplayerinfo.CommonPlayerInfo(player_id=2544)

In [17]:
player_info.available_seasons.get_data_frame()

,SEASON_ID
0,12003
1,22003
2,12004
3,22004
4,32004
5,12005
6,22005
7,32005
8,42005
9,12006


In [18]:
# Returns all data in a normalized dictionary.
player_info.get_normalized_dict()

{'AvailableSeasons': [{'SEASON_ID': '12003'},
  {'SEASON_ID': '22003'},
  {'SEASON_ID': '12004'},
  {'SEASON_ID': '22004'},
  {'SEASON_ID': '32004'},
  {'SEASON_ID': '12005'},
  {'SEASON_ID': '22005'},
  {'SEASON_ID': '32005'},
  {'SEASON_ID': '42005'},
  {'SEASON_ID': '12006'},
  {'SEASON_ID': '22006'},
  {'SEASON_ID': '32006'},
  {'SEASON_ID': '42006'},
  {'SEASON_ID': '12007'},
  {'SEASON_ID': '22007'},
  {'SEASON_ID': '32007'},
  {'SEASON_ID': '42007'},
  {'SEASON_ID': '12008'},
  {'SEASON_ID': '22008'},
  {'SEASON_ID': '32008'},
  {'SEASON_ID': '42008'},
  {'SEASON_ID': '12009'},
  {'SEASON_ID': '22009'},
  {'SEASON_ID': '32009'},
  {'SEASON_ID': '42009'},
  {'SEASON_ID': '12010'},
  {'SEASON_ID': '22010'},
  {'SEASON_ID': '32010'},
  {'SEASON_ID': '42010'},
  {'SEASON_ID': '12011'},
  {'SEASON_ID': '22011'},
  {'SEASON_ID': '32011'},
  {'SEASON_ID': '42011'},
  {'SEASON_ID': '12012'},
  {'SEASON_ID': '22012'},
  {'SEASON_ID': '32012'},
  {'SEASON_ID': '42012'},
  {'SEASON_ID': '1

In [19]:
# Find teams by full name.
teams.find_teams_by_full_name('cav')

[{'abbreviation': 'CLE',
  'city': 'Cleveland',
  'full_name': 'Cleveland Cavaliers',
  'id': 1610612739,
  'nickname': 'Cavaliers',
  'state': 'Ohio',
  'year_founded': 1970}]

In [20]:
from nba_api.stats.endpoints import playercareerstats
# Anthony Davis
career = playercareerstats.PlayerCareerStats(player_id='203076')
career.get_data_frames()[0]

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,203076,2012-13,00,1610612740,NOH,20.0,64,60,1846.0,349,...,0.751,165,357,522,63,75,112,89,158,867
1,203076,2013-14,00,1610612740,NOP,21.0,67,66,2358.0,522,...,0.791,207,466,673,105,89,189,109,200,1394
2,203076,2014-15,00,1610612740,NOP,22.0,68,68,2455.0,642,...,0.805,173,523,696,149,100,200,95,141,1656
3,203076,2015-16,00,1610612740,NOP,23.0,61,61,2164.0,560,...,0.758,130,497,627,116,78,125,121,148,1481
4,203076,2016-17,00,1610612740,NOP,24.0,75,75,2708.0,770,...,0.802,172,712,884,157,94,167,181,168,2099
5,203076,2017-18,00,1610612740,NOP,25.0,75,75,2727.0,780,...,0.828,187,644,831,174,115,193,162,159,2110
6,203076,2018-19,00,1610612740,NOP,26.0,56,56,1850.0,530,...,0.794,174,498,672,218,88,135,112,132,1452
7,203076,2019-20,00,1610612747,LAL,26.0,39,39,1349.0,357,...,0.865,81,275,356,124,59,96,94,100,1032


## Sources
1. https://github.com/swar/nba_api/tree/master/docs/examples  
2. https://towardsdatascience.com/link-prediction-with-neo4j-part-2-predicting-co-authors-using-scikit-learn-78b42356b44c  
3. https://medium.com/@davidheffernan_99410/an-introduction-to-using-categorical-embeddings-ee686ed7e7f9  
4. https://medium.com/@george.drakos62/decoded-entity-embeddings-of-categorical-variables-in-neural-networks-1d2468311635  
5. https://pypi.org/project/entity-embeddings-categorical/  